In [1]:
import selenium #셀레니움
import time
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들 

In [16]:
import requests

api_url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
headers = {'Authorization': 'KakaoAK 26d93ad65db2da865d5f56472aa14e71'}
params = {'query': '애견동반'}

response = requests.get(api_url, headers=headers, params=params)
data = response.json()

In [17]:
data

{'documents': [{'address_name': '경기 가평군 상면 임초리 605',
   'category_group_code': 'AD5',
   'category_group_name': '숙박',
   'category_name': '여행 > 숙박 > 펜션',
   'distance': '',
   'id': '7991865',
   'phone': '010-5536-1022',
   'place_name': '가평에스엔제이 애견동반펜션',
   'place_url': 'http://place.map.kakao.com/7991865',
   'road_address_name': '경기 가평군 상면 임초밤안골로 216',
   'x': '127.36662448025696',
   'y': '37.75275009988455'},
  {'address_name': '경기 포천시 이동면 장암리 14',
   'category_group_code': 'AD5',
   'category_group_name': '숙박',
   'category_name': '여행 > 숙박 > 펜션',
   'distance': '',
   'id': '25118861',
   'phone': '010-5227-1803',
   'place_name': '산울터애견동반펜션',
   'place_url': 'http://place.map.kakao.com/25118861',
   'road_address_name': '경기 포천시 이동면 늠바위길 148-169',
   'x': '127.38816097852565',
   'y': '38.02569160173375'},
  {'address_name': '인천 강화군 화도면 장화리 285-5',
   'category_group_code': 'AD5',
   'category_group_name': '숙박',
   'category_name': '여행 > 숙박 > 펜션',
   'distance': '',
   'id': '12

### 내가 찾아야 하는 반려동물 키워드 
- 반려동물 카페
- 반려동물 호텔
- 반려동물 산책로
- 반려동물 미용실
- 반려동물 동반 가능 식당
- 반려동물 동물병원
- 반려동물 공원
- 반려동물 동반 가능 쇼핑몰
- 반려동물 펫샵

## "애견동반" keyword

In [124]:
def get_store_list(start_x,start_y,end_x,end_y,cnt=1):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '애견동반',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print(' 불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

In [125]:
app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
# 사각형 증가값 설정
jump_x = 0.2
jump_y = 0.2

# 최종 데이터가 담길 리스트
cute_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        cute_list_one = get_store_list(start_x,start_y,end_x,end_y)
        cute_list.extend(cute_list_one)
        start_y = end_y
        # API 호출 제한을 위해 0.2초 대기
        time.sleep(0.2)
    start_x = end_x
# cute_list

0
11
0
0
0
0
0
0
0
2
0
0
2
92
big data dividing...
22
paging...
22
36
paging...
36
paging...
36
48
big data dividing...
12
8
29
paging...
29
27
paging...
27
4
60
big data dividing...
24
paging...
24
42
paging...
42
paging...
42
0
0
0
0
0
0
2
19
paging...
19
19
paging...
19
0
2
0
39
paging...
39
paging...
39
45
paging...
45
paging...
45
0
0
0
0
2
15
24
paging...
24
4
2
0
33
paging...
33
paging...
33
41
paging...
41
paging...
41
0
0
0
2
3
2
1
19
paging...
19
7
0
23
paging...
23
51
big data dividing...
20
paging...
20
18
paging...
18
26
paging...
26
8
0
0
0
1
2
3
3
105
big data dividing...
9
8
64
big data dividing...
13
29
paging...
29
25
paging...
25
37
paging...
37
paging...
37
48
big data dividing...
28
paging...
28
1
34
paging...
34
paging...
34
6
37
paging...
37
paging...
37
0
0
0
0
0
0
0
1
4
1
9
13
0
0
0
0
0
0
0
1
0
1
2
3
0
0
0
0
0
0
0
1
1
29
paging...
29
9
14
0
0
0
0
0
0
0
6
49
big data dividing...
2
23
paging...
23
11
19
paging...
19
13
14
18
paging...
18


In [126]:
import pandas as pd
jj = pd.DataFrame(cute_list2)
jj2 = jj.drop_duplicates(['addr'],keep='first')
jj2.category.value_counts()

카페     3036
숙박     1324
         70
음식점      27
Name: category, dtype: int64

In [127]:
len(jj2)

4457

In [128]:
jj2['do'] = jj2.addr.str.split(' ').str[0]
jj2['si'] = jj2.addr.str.split(' ').str[1]
jj2['gu'] = jj2.addr.str.split(' ').str[2]
jj2['category'] = jj2['category'].replace('','기타')
jj2.to_csv('kakao_map_animal_애견동반.csv',index=False,encoding='cp949')

/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/1665675696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['do'] = jj2.addr.str.split(' ').str[0]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/1665675696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['si'] = jj2.addr.str.split(' ').str[1]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/1665675696.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

## "애견동반 숙소" keyword

In [81]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '애견동반 숙소',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print(' 불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

In [82]:
app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
cute_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        cute_list_one = get_store_list(start_x,start_y,end_x,end_y)
        cute_list.extend(cute_list_one)
        start_y = end_y
        # API 호출 제한을 위해 0.2초 대기
        time.sleep(0.2)
    start_x = end_x
# cute_list

79
big data dividing...
1
24
paging...
24
21
paging...
21
61
big data dividing...
21
paging...
21
14
27
paging...
27
11
2
1
3
2
4
29
paging...
29
99
big data dividing...
7
76
big data dividing...
65
big data dividing...
2
45
paging...
45
paging...
45
12
28
paging...
28
7
13
0
10
10
17
paging...
17
34
paging...
34
paging...
34
0
0
55
big data dividing...
16
paging...
16
0
24
paging...
24
31
paging...
31
paging...
31
29
paging...
29
3
6
10
4
16
paging...
16
6
40
paging...
40
paging...
40
22
paging...
22
10
0
0
0
2
10
8
18
paging...
18
15
12
17
paging...
17
166
big data dividing...
6
65
big data dividing...
2
8
32
paging...
32
paging...
32
37
paging...
37
paging...
37
9
115
big data dividing...
58
big data dividing...
3
42
paging...
42
paging...
42
5
12
32
paging...
32
paging...
32
16
paging...
16
31
paging...
31
paging...
31
21
paging...
21
0
0
0
0
55
big data dividing...
12
26
paging...
26
8
21
paging...
21
23
paging...
23
9
2
7
21
paging...
21
121
big data dividing...
73
big data divid

In [83]:
cute_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
# cute2_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in all_stores)))
for i in cute_list:
    plac = i['place_name']
    cat = i['category_group_name']
    cat_code = i['category_group_code']
    addr = i['address_name']
    road_addr = i['road_address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    cute_list2.append({'name':plac, 'category':cat, 'cat_code':cat_code, 'addr':addr,'road_addr':road_addr, 'x':x_list, 'y':y_list})
len(cute_list2)

1715

In [84]:
import pandas as pd
jj = pd.DataFrame(cute_list2)

In [85]:
jj2 = jj.drop_duplicates(['addr'],keep='first')
jj2.category.value_counts()

숙박     1334
         37
음식점       2
Name: category, dtype: int64

In [86]:
jj2[jj2.category == '음식점']

,name,category,cat_code,addr,road_addr,x,y
811,귀곡산장,음식점,FD6,경기 가평군 가평읍 산유리 742-1,경기 가평군 가평읍 분자골로 234,37.761168,127.493368
1005,고구마캠프,음식점,FD6,충북 청주시 상당구 명암동 12,충북 청주시 상당구 명암로 263,36.654655,127.524081


In [88]:
jj2['do'] = jj2.addr.str.split(' ').str[0]
jj2['si'] = jj2.addr.str.split(' ').str[1]
jj2['gu'] = jj2.addr.str.split(' ').str[2]
jj2['category'] = jj2['category'].replace('','기타')
jj2.to_csv('kakao_map_animal_애견동반숙소.csv',index=False,encoding='cp949')

/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/923672222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['do'] = jj2.addr.str.split(' ').str[0]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/923672222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['si'] = jj2.addr.str.split(' ').str[1]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/923672222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

## 산책로 키워드 검색

In [89]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '산책로',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print(' 불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

In [90]:
app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
cute_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        cute_list_one = get_store_list(start_x,start_y,end_x,end_y)
        cute_list.extend(cute_list_one)
        start_y = end_y
        # API 호출 제한을 위해 0.2초 대기
        time.sleep(0.2)
    start_x = end_x
# cute_list

256
big data dividing...
2
62
big data dividing...
0
0
24
paging...
24
46
big data dividing...
0
0
34
paging...
34
paging...
34
17
paging...
17
53
big data dividing...
0
36
paging...
36
paging...
36
0
34
paging...
34
paging...
34
196
big data dividing...
36
paging...
36
paging...
36
44
paging...
44
paging...
44
98
big data dividing...
30
paging...
30
15
36
paging...
36
paging...
36
62
big data dividing...
46
big data dividing...
26
paging...
26
8
33
paging...
33
paging...
33
20
paging...
20
26
paging...
26
32
paging...
32
paging...
32
26
paging...
26
46
big data dividing...
11
3
29
paging...
29
18
paging...
18
18
paging...
18
11
41
paging...
41
paging...
41
10
12
22
paging...
22
98
big data dividing...
7
46
big data dividing...
28
paging...
28
7
14
0
24
paging...
24
24
paging...
24
48
big data dividing...
7
15
0
31
paging...
31
paging...
31
63
big data dividing...
1
57
big data dividing...
8
28
paging...
28
23
paging...
23
30
paging...
30
0
8
0
0
209
big data dividing...
57
big data di

In [91]:
cute_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
# cute2_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in all_stores)))
for i in cute_list:
    plac = i['place_name']
    cat = i['category_group_name']
    cat_code = i['category_group_code']
    addr = i['address_name']
    road_addr = i['road_address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    cute_list2.append({'name':plac, 'category':cat, 'cat_code':cat_code, 'addr':addr,'road_addr':road_addr, 'x':x_list, 'y':y_list})
len(cute_list2)

6452

In [92]:
len(cute_list)

6452

In [93]:
import pandas as pd
jj = pd.DataFrame(cute_list2)

In [94]:
jj2 = jj.drop_duplicates(['addr'],keep='first')
jj2.category.value_counts()

숙박      2130
        1477
관광명소     925
문화시설      16
음식점       10
카페         8
주차장        5
약국         1
Name: category, dtype: int64

In [95]:
jj2.category.value_counts()

숙박      2130
        1477
관광명소     925
문화시설      16
음식점       10
카페         8
주차장        5
약국         1
Name: category, dtype: int64

In [96]:
jj2['do'] = jj2.addr.str.split(' ').str[0]
jj2['si'] = jj2.addr.str.split(' ').str[1]
jj2['gu'] = jj2.addr.str.split(' ').str[2]
jj2['category'] = jj2['category'].replace('','기타')
jj2.to_csv('kakao_map_animal_산책로.csv',index=False,encoding='cp949')

/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/613433676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['do'] = jj2.addr.str.split(' ').str[0]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/613433676.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['si'] = jj2.addr.str.split(' ').str[1]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/613433676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

## 반려동물 공원

In [105]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '반려동물 공원',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print(' 불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

In [106]:
app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
cute_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        cute_list_one = get_store_list(start_x,start_y,end_x,end_y)
        cute_list.extend(cute_list_one)
        start_y = end_y
        # API 호출 제한을 위해 0.2초 대기
        time.sleep(0.2)
    start_x = end_x
# cute_list

0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
10
0
0
0
0
0
0
1
1
3
2
11
2
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [108]:
cute_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
# cute2_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in all_stores)))
for i in cute_list:
    plac = i['place_name']
    cat = i['category_group_name']
    cat_code = i['category_group_code']
    addr = i['address_name']
    road_addr = i['road_address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    cute_list2.append({'name':plac, 'category':cat, 'cat_code':cat_code, 'addr':addr,'road_addr':road_addr, 'x':x_list, 'y':y_list})
len(cute_list2)

46

In [109]:
import pandas as pd
jj = pd.DataFrame(cute_list2)
jj2 = jj.drop_duplicates(['addr'],keep='first')
jj2.category.value_counts()

      41
병원     3
Name: category, dtype: int64

In [111]:
jj2['do'] = jj2.addr.str.split(' ').str[0]
jj2['si'] = jj2.addr.str.split(' ').str[1]
jj2['gu'] = jj2.addr.str.split(' ').str[2]
jj2['category'] = jj2['category'].replace('','기타')
jj2.to_csv('kakao_map_animal_반려동물공원.csv',index=False,encoding='cp949')

/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/3018297710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['do'] = jj2.addr.str.split(' ').str[0]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/3018297710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['si'] = jj2.addr.str.split(' ').str[1]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/3018297710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

## 반려견동반

In [116]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '반려견동반',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print(' 불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

In [119]:
app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
cute_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        cute_list_one = get_store_list(start_x,start_y,end_x,end_y)
        cute_list.extend(cute_list_one)
        start_y = end_y
        # API 호출 제한을 위해 0.2초 대기
        time.sleep(0.2)
    start_x = end_x
# cute_list

133
big data dividing...
1
31
paging...
31
paging...
31
34
paging...
34
paging...
34
107
big data dividing...
30
paging...
30
17
paging...
17
48
big data dividing...
25
paging...
25
11
6
20
paging...
20
29
paging...
29
6
2
29
paging...
29
2
5
31
paging...
31
paging...
31
110
big data dividing...
7
83
big data dividing...
68
big data dividing...
3
47
big data dividing...
29
paging...
29
27
paging...
27
27
paging...
27
11
13
30
paging...
30
8
14
2
11
14
30
paging...
30
52
big data dividing...
0
49
big data dividing...
4
24
paging...
24
13
20
paging...
20
0
2
0
0
97
big data dividing...
25
paging...
25
0
47
big data dividing...
24
paging...
24
10
10
9
49
big data dividing...
21
paging...
21
3
15
21
paging...
21
46
big data dividing...
23
paging...
23
24
paging...
24
0
0
4
9
121
big data dividing...
12
86
big data dividing...
12
14
35
paging...
35
paging...
35
46
big data dividing...
37
paging...
37
paging...
37
3
8
4
6
21
paging...
21
43
paging...
43
paging...
43
26
paging...
26
46
big da

In [120]:
cute_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
# cute2_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in all_stores)))
for i in cute_list:
    plac = i['place_name']
    cat = i['category_group_name']
    cat_code = i['category_group_code']
    addr = i['address_name']
    road_addr = i['road_address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    cute_list2.append({'name':plac, 'category':cat, 'cat_code':cat_code, 'addr':addr,'road_addr':road_addr, 'x':x_list, 'y':y_list})
len(cute_list2)

7265

In [121]:
import pandas as pd
jj = pd.DataFrame(cute_list2)
jj2 = jj.drop_duplicates(['addr'],keep='first')
jj2.category.value_counts()

카페     3036
숙박     1324
         70
음식점      27
Name: category, dtype: int64

In [123]:
jj2['do'] = jj2.addr.str.split(' ').str[0]
jj2['si'] = jj2.addr.str.split(' ').str[1]
jj2['gu'] = jj2.addr.str.split(' ').str[2]
jj2['category'] = jj2['category'].replace('','기타')
jj2.to_csv('kakao_map_animal_반려견동반.csv',index=False,encoding='cp949')

/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/870249315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['do'] = jj2.addr.str.split(' ').str[0]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/870249315.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['si'] = jj2.addr.str.split(' ').str[1]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/870249315.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

## 애견동반 식당

In [129]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '애견동반 식당',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print(' 불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

In [131]:
app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
cute_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        cute_list_one = get_store_list(start_x,start_y,end_x,end_y)
        cute_list.extend(cute_list_one)
        start_y = end_y
        # API 호출 제한을 위해 0.2초 대기
        time.sleep(0.2)
    start_x = end_x
# cute_list

48
big data dividing...
1
12
16
paging...
16
34
paging...
34
paging...
34
2
0
15
2
0
1
17
paging...
17
12
20
paging...
20
0
0
28
paging...
28
18
paging...
18
0
3
43
paging...
43
paging...
43
9
6
15
174
big data dividing...
8
6
71
big data dividing...
13
12
12
47
big data dividing...
19
paging...
19
10
8
20
paging...
20
103
big data dividing...
20
paging...
20
21
paging...
21
37
paging...
37
paging...
37
34
paging...
34
paging...
34
265
big data dividing...
84
big data dividing...
12
32
paging...
32
paging...
32
12
40
paging...
40
paging...
40
167
big data dividing...
28
paging...
28
79
big data dividing...
48
big data dividing...
4
8
12
42
paging...
42
paging...
42
31
paging...
31
paging...
31
33
paging...
33
paging...
33
10
43
paging...
43
paging...
43
35
paging...
35
paging...
35
7
28
paging...
28
0
0
0
0
0
2
5
8
47
big data dividing...
1
2
11
37
paging...
37
paging...
37
55
big data dividing...
5
15
33
paging...
33
paging...
33
3
163
big data dividing...
37
paging...
37
paging...
37

In [132]:
cute_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
# cute2_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in all_stores)))
for i in cute_list:
    plac = i['place_name']
    cat = i['category_group_name']
    cat_code = i['category_group_code']
    addr = i['address_name']
    road_addr = i['road_address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    cute_list2.append({'name':plac, 'category':cat, 'cat_code':cat_code, 'addr':addr,'road_addr':road_addr, 'x':x_list, 'y':y_list})
len(cute_list2)

2237

In [133]:
import pandas as pd
jj = pd.DataFrame(cute_list2)
jj2 = jj.drop_duplicates(['addr'],keep='first')
jj2.category.value_counts()

음식점    1235
카페      417
          9
숙박        1
Name: category, dtype: int64

In [134]:
jj2['do'] = jj2.addr.str.split(' ').str[0]
jj2['si'] = jj2.addr.str.split(' ').str[1]
jj2['gu'] = jj2.addr.str.split(' ').str[2]
jj2['category'] = jj2['category'].replace('','기타')
jj2.to_csv('kakao_map_animal_애견동반식당.csv',index=False,encoding='cp949')

/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/147475332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['do'] = jj2.addr.str.split(' ').str[0]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/147475332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jj2['si'] = jj2.addr.str.split(' ').str[1]
/var/folders/qr/778czgkd1p7ch8wn4f6dmdkh0000gn/T/ipykernel_1670/147475332.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

## 애견동반카페 키워드 찾기

In [135]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '애견동반 카페',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print(' 불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

In [136]:
app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
cute_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        cute_list_one = get_store_list(start_x,start_y,end_x,end_y)
        cute_list.extend(cute_list_one)
        start_y = end_y
        # API 호출 제한을 위해 0.2초 대기
        time.sleep(0.2)
    start_x = end_x
# cute_list

57
big data dividing...
0
7
14
48
big data dividing...
10
3
22
paging...
22
18
paging...
18
4
1
26
paging...
26
0
1
3
11
12
21
paging...
21
0
0
48
big data dividing...
11
0
27
paging...
27
21
paging...
21
21
paging...
21
1
4
112
big data dividing...
10
87
big data dividing...
13
14
35
paging...
35
paging...
35
46
big data dividing...
37
paging...
37
paging...
37
3
8
4
3
15
38
paging...
38
paging...
38
11
40
paging...
40
paging...
40
414
big data dividing...
13
25
paging...
25
166
big data dividing...
7
22
paging...
22
21
paging...
21
144
big data dividing...
31
paging...
31
paging...
31
36
paging...
36
paging...
36
31
paging...
31
paging...
31
80
big data dividing...
30
paging...
30
11
36
paging...
36
paging...
36
46
big data dividing...
11
6
31
paging...
31
paging...
31
32
paging...
32
paging...
32
234
big data dividing...
39
paging...
39
paging...
39
60
big data dividing...
11
31
paging...
31
paging...
31
18
paging...
18
24
paging...
24
75
big data dividing...
13
10
52
big data divid

In [137]:
cute_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
# cute2_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in all_stores)))
for i in cute_list:
    plac = i['place_name']
    cat = i['category_group_name']
    cat_code = i['category_group_code']
    addr = i['address_name']
    road_addr = i['road_address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    cute_list2.append({'name':plac, 'category':cat, 'cat_code':cat_code, 'addr':addr,'road_addr':road_addr, 'x':x_list, 'y':y_list})
len(cute_list2)

5476

In [138]:
import pandas as pd
jj = pd.DataFrame(cute_list2)
jj2 = jj.drop_duplicates(['addr'],keep='first')
jj2.category.value_counts()

카페     3132
음식점      27
         22
숙박        2
Name: category, dtype: int64

In [ ]:
jj2['do'] = jj2.addr.str.split(' ').str[0]
jj2['si'] = jj2.addr.str.split(' ').str[1]
jj2['gu'] = jj2.addr.str.split(' ').str[2]
jj2['category'] = jj2['category'].replace('','기타')
jj2.to_csv('kakao_map_animal_애견동반카페.csv',index=False,encoding='cp949')